# Python Intro - Final Exercise


In [1]:
import pyodbc
import pandas as pd
import numpy as np

In [2]:
### In windows, Using a ODBC DNS (predefined connection name)
conn = pyodbc.connect('DSN=COLLEGE;Trusted_Connection=yes;')


In [3]:
### Get the students table
students = pd.read_sql("SELECT * FROM Students;",conn) 
classrooms = pd.read_sql("SELECT * FROM Classrooms;",conn) 
teachers = pd.read_sql("SELECT * FROM Teachers;",conn) 
courses = pd.read_sql("SELECT * FROM Courses;",conn) 
departments = pd.read_sql("SELECT * FROM Departments;",conn) 

In [4]:
#students
students_classrooms = pd.merge(students, classrooms, how='inner', on='StudentId')
students_classrooms_courses = pd.merge(students_classrooms, courses, how='inner', on='CourseId')
students_classrooms_courses_departments = pd.merge(students_classrooms_courses, departments, how='inner', left_on='DepartmentID', right_on='DepartmentId')

# Questions
 
 
## Q1. Count the number of students on each department

In [5]:
department_student_unique = students_classrooms_courses_departments[['DepartmentName','StudentId']].drop_duplicates()
dsu = department_student_unique.groupby('DepartmentName').agg({'StudentId':['count']})
dsu.columns = ['Students']
dsu

,Students
DepartmentName,
Arts,137
English,84
Science,200
Sport,81


## Q2. How many students have each course of the English department and the total number of students in the department?

In [6]:
### filter students by department==English
eng_studs = students_classrooms_courses_departments.loc[students_classrooms_courses_departments['DepartmentName'] == 'English']
### group by course and count inside the group
stud_course = eng_studs.groupby('CourseName').agg({'StudentId':['count']})
### Rename aggregated column
stud_course.columns = ['Students']
### count the distinct students in department
distinct_studs = eng_studs.StudentId.drop_duplicates().count()
### add total row
stud_course.loc['Total']=distinct_studs
stud_course

,Students
CourseName,
Advanced English,24
English Begginers,32
Proffesional English,38
Total,84


## Q3. How many small (<22 students) and large (22+ students) classrooms are needed for the Science department?

In [7]:
### filter students by department==English
sc_studs = students_classrooms_courses_departments.loc[students_classrooms_courses_departments['DepartmentName'] == 'Science']
classrooms = sc_studs.groupby('CourseName').agg({'StudentId':['count']})
classrooms.columns = ['Students']
classrooms['classroom_size'] = classrooms.apply(lambda row: 'Big classrooms' if row.Students >= 22 else 'Small classrooms', axis=1)
res = classrooms.groupby('classroom_size').agg({'Students':['count']})
res.columns = ['num_classrooms']
res

,num_classrooms
classroom_size,
Big classrooms,10
Small classrooms,6


## Q4. A feminist student claims that there are more male than female in the College. Justify if the argument is correct

In [8]:
studs_gender = students.groupby('Gender').agg({'StudentId':['count']})
studs_gender.columns = ['num_students']
studs_gender

,num_students
Gender,
F,165
M,115


## Q5. For which courses the percentage of male/female students is over 70%?

In [9]:
### group by course and gender, aggregate num of students
courses_gender_aggr = students_classrooms_courses.groupby(['CourseId','CourseName','Gender']).agg({'StudentId':['count']})
### calc percentage of students per course and gender
courses_gender_prc = courses_gender_aggr.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))
courses_gender_prc.columns = ['studentsPercentage']
### filter > 70%
res = courses_gender_prc[courses_gender_prc.studentsPercentage > 70]

res

,,,studentsPercentage
CourseId,CourseName,Gender,
22.0,Sculpture,F,70.833333
29.0,Tenis,F,72.222222


## Q6. For each department, how many students passed with a grades over 80?

In [10]:
over80 = students_classrooms_courses_departments[students_classrooms_courses_departments.degree > 80]
over80unique = over80.drop_duplicates(subset=['DepartmentId', 'StudentId'], keep='first')
over80res = over80unique.groupby("DepartmentName").agg({'StudentId':['count']})
over80res_totals = pd.merge(over80res, dsu, how='inner', on='DepartmentName')
over80res_totals.columns = ['students_80','total_students']
over80res_totals['students_80_pct'] = over80res_totals.students_80 / over80res_totals.total_students * 100
over80res_totals

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


,students_80,total_students,students_80_pct
DepartmentName,,,
Arts,61,137,44.525547
English,40,84,47.619048
Science,122,200,61.000000
Sport,39,81,48.148148


## Q7. For each department, how many students passed with a grades under 60?

In [11]:
under60 = students_classrooms_courses_departments[students_classrooms_courses_departments.degree < 60]
under60unique = under60.drop_duplicates(subset=['DepartmentId', 'StudentId'], keep='first')
under60res = under60unique.groupby("DepartmentName").agg({'StudentId':['count']})
under60res_totals = pd.merge(under60res, dsu, how='inner', on='DepartmentName')
under60res_totals.columns = ['students_60','total_students']
under60res_totals['students_60_pct'] = under60res_totals.students_60 / under60res_totals.total_students * 100
under60res_totals

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


,students_60,total_students,students_60_pct
DepartmentName,,,
Arts,3,137,2.189781
English,2,84,2.380952
Science,7,200,3.500000
Sport,1,81,1.234568


## Q8. Rate the teachers by their average student's grades (in descending order).

In [12]:
students_classrooms_courses_teachers = pd.merge(students_classrooms_courses, teachers, how='inner', on='TeacherId')
students_classrooms_courses_teachers['Teacher'] = students_classrooms_courses_teachers['FirstName_y'] + ' ' + students_classrooms_courses_teachers['LastName_y']
res_teachers = students_classrooms_courses_teachers.groupby('Teacher').agg({'degree':['mean']})
res_teachers.columns = ['avg_degrees']
res_teachers.sort_values(by=['avg_degrees'], ascending=False)


,avg_degrees
Teacher,
Darick Hess,81.432703
Jordyn Baron,80.729494
Heather Metcalf,80.440242
Zachary Healy,79.365867
Kaylea Monzingo,79.365537
Daniel Bland,78.908281
Keegan Price,78.814631
Conor Mckittrick,78.699492
Maxwell Romkee,78.673509


## Q9. Create a dataframe showing the courses, departments they are associated with, the teacher in each course, and the number of students enrolled in the course (for each course, department and teacher show the names).

In [13]:
sccd = students_classrooms_courses_departments.copy()
del sccd['FirstName']
del sccd['LastName']
students_classrooms_courses_departments_teachers = pd.merge(sccd, teachers, how='inner', on='TeacherId')

res1 = students_classrooms_courses_departments_teachers\
.groupby(['CourseId','CourseName','DepartmentName','FirstName', 'LastName'])\
.agg({'StudentId':['count']})

res1.columns = ['num_students']
res1

,,,,,num_students
CourseId,CourseName,DepartmentName,FirstName,LastName,
1.0,English Begginers,English,Conor,Mckittrick,32
2.0,Advanced English,English,Conor,Mckittrick,24
3.0,Proffesional English,English,Conor,Mckittrick,38
4.0,Mathematics 101,Science,Amanda,Dworkin,18
5.0,Matematics Advanced,Science,Alexander,Watts,10
6.0,Algebra,Science,Alexander,Watts,12
7.0,Geometry,Science,Alexander,Watts,14
8.0,Chemistry,Science,Jeffrey,Batzel,24
9.0,Physics,Science,Alexander,Watts,12


## Q10. Create a dataframe showing the students, the number of courses they take, the average of the grades per class, and their overall average (for each student show the student name).

In [26]:
res10 = pd.pivot_table(students_classrooms_courses_departments, values='degree', index=['StudentId', 'FirstName','LastName'],
                    columns=['DepartmentName'], aggfunc=np.mean)


res10

,,DepartmentName,Arts,English,Science,Sport
StudentId,FirstName,LastName,,,,
1.0,Alexandria,Heller,86.677535,NaN,79.557508,NaN
2.0,Cassie,Liska,79.542738,NaN,78.867115,NaN
3.0,Hayley,Frank,NaN,83.972245,76.204427,NaN
4.0,Maxwell,Kotch,69.356144,70.929801,74.157066,NaN
5.0,Dylan,Boehm,64.494845,NaN,100.000000,65.661106
...,...,...,...,...,...,...
275.0,Elise,Ohler,94.702880,71.617451,80.897496,NaN
276.0,Dylan,Bannister,NaN,NaN,76.784735,NaN
277.0,Bryan,Medina,78.849330,NaN,81.298837,NaN
